In [1]:
from pprint import pprint
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

True

# Telemetry

Run Phoenix server with
```
python -m phoenix.server.main serve
```
or
```
phoenix serve
```

In [2]:
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

from phoenix.otel import register

endpoint = "http://0.0.0.0:6006/v1/traces"

# trace_provider = TracerProvider() # this is from HF page.
trace_provider = register(
    project_name="smolagents-project-cd-ep21",
    endpoint=endpoint,
    set_global_tracer_provider=False
)
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

/Users/pisek/miniconda3/envs/smol/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: smolagents-project-cd-ep21
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://0.0.0.0:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.



In [3]:
from smolagents import OpenAIServerModel

model_openai = OpenAIServerModel(
    model_id="gpt-4o-mini",
    api_base="https://api.openai.com/v1",
    api_key=os.environ["OPENAI_API_KEY"],
)

In [4]:
from smolagents import Tool, tool

@tool
def virtual_cat(interact: str) -> str:
    """
    This is a tool that the user can interact with a virtual cat. The user will get a response from a cat.
    Usage example: response = virtual_cat(interact="give food to cat")

    Args:
        interact: interaction the user wants to do with the cat
    """
    return "~Meow, Meol, Meaw~"


class VirtualDog(Tool):
    name = "virtual_dog"
    description = """This is a tool that the user can interact with a virtual dog. The user will get a response from a dog.
    Usage example: response = virtual_dog(interact="give food to dog")
    """
    inputs = {"interact": {"type": "string", "description": "interaction the user wants to do with the dog"}}
    output_type = "string"

    def forward(self, interact: str) -> str:
        return "~woof, woof, wooooooof~"

In [5]:
from smolagents import CodeAgent

cat_agent = CodeAgent(tools=[virtual_cat], model=model_openai, 
                      name="cat_agent",
                      description="Send interactions and get response from cats. Use this agent by asking it to use tools and specifying \"interact\""
                      )
dog_agent = CodeAgent(tools=[VirtualDog()], model=model_openai,
                      name="dog_agent",
                      description="send interactions and get response from dogs. Use this agent by asking it to use tools and specifying \"interact\""
)

manager_agent = CodeAgent(
    tools=[], model=model_openai, managed_agents=[cat_agent, dog_agent], max_steps=6
)

In [6]:
manager_agent.run("throw a ball for the dog and pur pur with the cat")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ throw a ball for the dog and pur pur with the cat                                                               │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  dog_agent(task="throw a ball for the dog")                                                                       
  cat_agent(task="pur pur with the cat")                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭────────────────────────────────────────────── New run - dog_agent ──────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'dog_agent'.                                                                       │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ throw a ball for the dog                                                                                        │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o-mini ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  results = {                                                                                                      
      "cat": {                                                                                                     
          "short_summary": "The cat responded with playful sounds.",                                               
          "detailed_summary": "When I interacted with the virtual cat using the phrase 'pur pur', it responded     
  with sounds similar to '~Meow, Meol, Meaw~', indicating a playful and relaxed demeanor.",                        
      },                                                                                                           
      "dog": {                                                                                                     
          "short_summary": "The dog responded excitedly to the ball being thrown.",                                
          "detailed_summary": "Upon throwing the ball for the dog, it expressed excitement with a series of        
  'woof' sounds, indicating a playful and enthusiastic reaction."                                                  
      }                                                                                                            
  }                                                                                                                
                                                                                                                   
  final_answer(results)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'cat': {'short_summary': 'The cat responded with playful sounds.', 'detailed_summary': "When I
interacted with the virtual cat using the phrase 'pur pur', it responded with sounds similar to '~Meow, Meol, 
Meaw~', indicating a playful and relaxed demeanor."}, 'dog': {'short_summary': 'The dog responded excitedly to the 
ball being thrown.', 'detailed_summary': "Upon throwing the ball for the dog, it expressed excitement with a series
of 'woof' sounds, indicating a playful and enthusiastic reaction."}}

[Step 2: Duration 2.52 seconds| Input tokens: 7,374 | Output tokens: 596]

{'cat': {'short_summary': 'The cat responded with playful sounds.',
  'detailed_summary': "When I interacted with the virtual cat using the phrase 'pur pur', it responded with sounds similar to '~Meow, Meol, Meaw~', indicating a playful and relaxed demeanor."},
 'dog': {'short_summary': 'The dog responded excitedly to the ball being thrown.',
  'detailed_summary': "Upon throwing the ball for the dog, it expressed excitement with a series of 'woof' sounds, indicating a playful and enthusiastic reaction."}}

NameError: name 'response' is not defined